# Load point values

This is a demo that queries a list of points and orders by risk according to Wildfire Risk based on the [Wildfire Forecast](https://data.spatiafi.com/dataset/36) dataset.  We start by loading our App Credentials, and then using the [point](https://docs.spatiafi.com/api/#operation/point_api_point__lon___lat__get) endpoint to fetch fire risk values at the points of interest.

---

Install `spatiafi` (if not already installed):

In [ ]:
!sudo apt -y install libgeos-dev

In [ ]:
%pip install spatiafi cartopy xarray pandas

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spatiafi
import xarray as xr

In [ ]:
session = spatiafi.get_session()

In [ ]:
session

## Point Value
This section takes a list of example asset locations (lats/lons) and queries the day 1 forecast of FWI at each location. This should give us an understanding of the near-term fire risk for our assets

In [ ]:
lon = ["-104.18", "-92.88", "-113.46", "-122.33", "-74.00", "-89.44", "25.66", "137.41"]
lat = ["55.48", "55.74", "56.99", "47.61", "40.71", "31.01", "23.41", "-24.57"]

values = np.zeros(len(lat))

for i in range(len(lon)):
    url = "https://api.spatiafi.com/api/point/" + lon[i] + "," + lat[i]

    query = {"item_id": "ce-cfsv2-era5-fwi-forecast-day01", "bidx": "1"}

    not_done = True
    while not_done:
        response = session.get(url, params=query)

        if response.status_code != int(500):
            not_done = False
            data = response.json()
            values[i] = data["values"][0]

    print(data)

We now plot these values so that we can more clearly see which assets are of greatest risk.

In [ ]:
lon_double = [float(x) for x in lon]
lat_double = [float(x) for x in lat]

data = xr.DataArray(values, dims=("points",), coords={"points": range(len(values))})

# Add latitude and longitude as coordinates
data = data.assign_coords(
    latitude=("points", lat_double), longitude=("points", lon_double)
)

fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the data using scatter plot
plt.scatter(
    data["longitude"],
    data["latitude"],
    c=data,
    cmap="viridis",
    transform=ccrs.PlateCarree(),
)

# Add coastlines
ax.coastlines()

# Show the plot
plt.show()

We can see that our assets with the largest FWI June 12th forecast are in Australia and North Africa, as should be expected. There are also relatively high values in parts of Canada, consistent with the large wildfires over the past couple of weeks.

In [ ]:
df = pd.DataFrame({"Lon": lon, "Lat": lat, "FWI": values})

df.sort_values("FWI", ascending=False)

We can also easily save the data out as a pandas DataFrame, allowing us to do things like sort by FWI to see the assets at greatest risk.

## Point Values Over Time
We now include an example that queries timeseries from the same point values. This function can be used to, for example, track trends in the FWI to see if there will be assets we should be concerned about in a few days' time. Here we use it to query the next 5 days of data at each location and calculate the trend to see if there is a notable increase or decrease.

In [ ]:
values_array = np.zeros((5, len(lat)))
values_one = np.zeros(5)
slopes = np.zeros(len(lat))

for i in range(len(lon)):
    url = "https://api.spatiafi.com/api/timeseries/point/" + lon[i] + "," + lat[i]

    query = {
        "coll_id": "climate-engine-cfsv2-era5-land-fire-weather-index-forecasts",
        "datetime": "2023-07-17/2023-07-23",
        "bidx": "1",
    }

    not_done = True
    while not_done:
        response = session.get(url, params=query)

        if response.status_code != int(500):
            not_done = False
            data = response.json()

    index = 4
    for key in data:
        values_one[index] = data[key]["values"][0]
        index = index - 1

    slopes[i] = np.polyfit(np.linspace(1, 5, 5), values_one, 1)[0]

    # Convert the list of values to a NumPy array
    values_array[:, i] = np.array(values_one)

    print(i)

In [ ]:
data = xr.DataArray(slopes, dims=("points",), coords={"points": range(len(slopes))})

# Add latitude and longitude as coordinates
data = data.assign_coords(
    latitude=("points", lat_double), longitude=("points", lon_double)
)

fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the data using scatter plot
plt.scatter(
    data["longitude"],
    data["latitude"],
    c=data,
    cmap="bwr",
    transform=ccrs.PlateCarree(),
)

# Add coastlines
ax.coastlines()

# Show the plot
plt.show()

From this, we can see that some of our assets (such as the east-most point in Canada and in the Pacific Northwest) indicate positive trends in their FWI (i.e., an upcoming increase in fire risk), while others exhibit a decline.